<a href="https://colab.research.google.com/github/Gurbnova/Edmonton-House-Predictions/blob/main/Edmonton_House_Predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import matplotlib
matplotlib.rcParams["figure.figsize"] = (20, 10)

df1 = pd.read_csv("edmonton_housing_data_Feb15_2022.csv.zip", compression="zip")

#
df1.head()

,House Description,Price,Bedrooms,Bathrooms,Full Baths,Square Footage,Acres,Year Built,Type,Sub-Type,...,Bedrooms Above Grade,# of Garages,Garages,Half Baths,Fireplaces,Zoning,Lot Description,HOA Fees,HOA Fees Freq.,Condo Fee
0,"Rare property on the market, house converted t...","$399,990",1,1.0,1.0,787,0.12,1948,Single Family,Residential Detached Single Family,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,This fantastic family home has an option to As...,"$357,000",4,2.0,2.0,929,0.15,1954,Single Family,Residential Detached Single Family,...,2.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"At over 1150 square feet, this Allendale 3+2 b...","$499,600",5,2.0,2.0,"1,161",0.14,1959,Single Family,Residential Detached Single Family,...,3.0,1,7.32x4.27,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Gorgeous upgrades and a lower level suite in C...,"$398,800",4,3.0,3.0,"1,217",0.15,1976,Single Family,Residential Detached Single Family,...,3.0,2,24x26,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AMAZING VALUE! Welcome home to this well-maint...,"$279,900",3,2.0,1.0,"1,160",0.00,1979,Single Family,Residential Detached Single Family,...,3.0,2,NaN,1.0,"Gas, Oak Surround",NaN,NaN,NaN,NaN,NaN


In [57]:
df1.shape

(1697, 52)

In [58]:
# Removing unnecessary  columns
df2 = df1.loc[:, ['Price', 'Bedrooms', 'Bathrooms', 'Square Footage', 'Community']]

df2.head()

,Price,Bedrooms,Bathrooms,Square Footage,Community
0,"$399,990",1,1.0,787,Glenwood
1,"$357,000",4,2.0,929,Woodcroft
2,"$499,600",5,2.0,"1,161",Allendale
3,"$398,800",4,3.0,"1,217",Caernarvon
4,"$279,900",3,2.0,"1,160",Newton


In [59]:
# Checking for NAs
df2.isnull().sum()

Price             0
Bedrooms          0
Bathrooms         0
Square Footage    0
Community         0
dtype: int64

In [60]:
# Converting string numbers to integers
df2['Bathrooms'] = df2['Bathrooms'].astype(int)

df2['Bathrooms'].unique() # For checking purposes

array([ 1,  2,  3,  0,  4,  6,  5,  7,  8, 12,  9])

In [61]:
# Converting string decimals to floats
def is_float(x):
  try:
    float(x)
  except:
    return False
  return True

df2[~df2['Square Footage'].apply(is_float)]

,Price,Bedrooms,Bathrooms,Square Footage,Community
2,"$499,600",5,2,"1,161",Allendale
3,"$398,800",4,3,"1,217",Caernarvon
4,"$279,900",3,2,"1,160",Newton
6,"$314,850",3,2,"1,119",Kildare
9,"$219,900",3,1,"1,109",Boyle Street
...,...,...,...,...,...
1692,"$399,900",6,2,"1,227",Lauderdale
1693,"$2,795,000",6,6,"4,593",Strathcona
1694,"$239,900",6,2,"1,783",Alberta Avenue
1695,"$839,900",4,9,"3,445",Newton


In [62]:
# Four digited Square footages have an unnecessary comma that disables float conversion

def str_to_float(x):
  if not is_float(x):
    return float(x.replace(',', ''))
  return float(x)

df2['Square Footage'] = df2['Square Footage'].apply(str_to_float)

df2[~df2['Square Footage'].apply(is_float)]  # For checking purposes

,Price,Bedrooms,Bathrooms,Square Footage,Community


In [63]:
# Converting (dollar signed) prices to integers (also with commas)

def price_to_int(x):
  return int(str_to_float(x[1:])) # Using str_to_float to get rid of commas

df2['Price'] = df2['Price'].apply(price_to_int)

In [64]:
df3 = df2.copy()

# Creating price per square foot feature to clean outliers
df3['Price Per Sqft'] = df3['Price'] / df3['Square Footage']

df3.head()

,Price,Bedrooms,Bathrooms,Square Footage,Community,Price Per Sqft
0,399990,1,1,787.0,Glenwood,508.246506
1,357000,4,2,929.0,Woodcroft,384.284177
2,499600,5,2,1161.0,Allendale,430.318691
3,398800,4,3,1217.0,Caernarvon,327.691044
4,279900,3,2,1160.0,Newton,241.293103


In [65]:
len(df3['Community'].unique())

150

In [66]:
# Since our Community column does not have too much dimensionality, we will keep it as is

Community
Queen Mary Park            54
Oliver                     52
Westmount                  46
Strathcona                 45
Garneau                    40
                           ..
Mill Creek Ravine North     1
Duggan                      1
Sherbrooke                  1
Mcqueen                     1
Dunluce                     1
Name: Community, Length: 150, dtype: int64

In [72]:
# A typical threshold for square footage per bedroom is 300 sqft
# So we will look at the cases where it is lower than that threshold and get rid of them (unusual datapoints)
df3[df3['Square Footage']/df3['Bedrooms']<300].head()

,Price,Bedrooms,Bathrooms,Square Footage,Community,Price Per Sqft
1,357000,4,2,929.0,Woodcroft,384.284177
2,499600,5,2,1161.0,Allendale,430.318691
7,189900,3,1,694.0,Belvedere,273.631124
8,238888,4,0,926.0,Boyle Street,257.978402
11,419000,3,2,815.0,Dovercourt,514.110429


In [74]:
df3.shape

(1697, 6)

In [75]:
df4 = df3[~(df3['Square Footage']/df3['Bedrooms']<300)]
df4.shape

(1330, 6)

In [76]:
# Now we will get rid of unusual datapoints according to price per square foot

df4['Price Per Sqft'].describe()

count    1330.000000
mean      289.937738
std       137.139329
min        34.461009
25%       195.886021
50%       271.499086
75%       358.184422
max      1406.666667
Name: Price Per Sqft, dtype: float64

In [79]:
# We will assume our price per sqft column has a normal distribution and remove anything beyond one std away from the mean

def remove_pps_outliers(df):
  df_out = pd.DataFrame()

  for key, subdf in df.groupby('Community'):
    m = np.mean(subdf['Price Per Sqft'])
    st = np.std(subdf['Price Per Sqft'])
    reduced_df = subdf[(subdf['Price Per Sqft']>(m-st)) & (subdf['Price Per Sqft']<=(m+st))]
    df_out = pd.concat([df_out, reduced_df], ignore_index=True)

  return df_out

df5 = remove_pps_outliers(df4)
df

,Price,Bedrooms,Bathrooms,Square Footage,Community,Price Per Sqft
0,155000,3,2,962.0,Abbottsfield,161.122661
1,162800,3,2,956.0,Abbottsfield,170.292887
2,215000,2,1,754.0,Alberta Avenue,285.145889
3,224995,2,1,704.0,Alberta Avenue,319.595170
4,149900,1,1,672.0,Alberta Avenue,223.065476
...,...,...,...,...,...,...
890,899900,4,4,1753.0,Windsor Park,513.348545
891,695000,2,3,1329.0,Windsor Park,522.949586
892,629900,2,3,1082.0,Windsor Park,582.162662
893,129000,1,1,661.0,Woodcroft,195.158850
